In [5]:
import cv2
import numpy as np
import os
import SimpleITK as itk
import pydicom
import shutil

In [6]:
def image_erode(image, size):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size, size))
    image = cv2.erode(image, kernel)
    return image


def image_expand(image, size):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (size, size))
    image = cv2.dilate(image, kernel)
    return image

'''
    此ｂｌｏｃｋ封装一些常见的操作
'''
# 读取单个DICOM文件
def read_dicom_file(file_name):
    header = pydicom.read_file(file_name)
    image = header.pixel_array
    image = header.RescaleSlope * image + header.RescaleIntercept
    return image
# 读取mhd文件
def read_mhd_image(file_path):
    header = itk.ReadImage(file_path)
    image = itk.GetArrayFromImage(header)
    return np.array(image)


# 保存mhd文件
def save_mhd_image(image, file_name):
    header = itk.GetImageFromArray(image)
    itk.WriteImage(header, file_name)

In [7]:
def ExtractSLROI(sl_path, roi_save_dir):
    sl_names = os.listdir(sl_path)
    for sl_name in sl_names:
        print '-'*15, 'processing %s' % sl_name, '-'*15
        roi_save_path = os.path.join(roi_save_dir, sl_name)
        if not os.path.exists(roi_save_path):
            os.mkdir(roi_save_path)
        for phase in ['NC', 'ART', 'PV']:
            image = read_mhd_image(os.path.join(sl_path, sl_name, phase+'_Image.mhd'))
            mask = read_mhd_image(os.path.join(sl_path, sl_name, phase+'_Mask.mhd'))
            # image[mask == 0] = 0
            [ys, xs] = np.where(mask != 0)
            border = 5
            roi_image = image[np.min(ys) - border:np.max(ys)+border, np.min(xs)-border: np.max(xs)+border]
            save_path = os.path.join(roi_save_path, phase+'_ROI.mhd')
            save_mhd_image(roi_image, save_path)

# 将非ROI的区域置为0
def ExtractSLROIZero(sl_path, roi_save_dir):
    sl_names = os.listdir(sl_path)
    for sl_name in sl_names:
        print '-'*15, 'processing %s' % sl_name, '-'*15
        roi_save_path = os.path.join(roi_save_dir, sl_name)
        if not os.path.exists(roi_save_path):
            os.mkdir(roi_save_path)
        for phase in ['NC', 'ART', 'PV']:
            image = read_mhd_image(os.path.join(sl_path, sl_name, phase+'_Image.mhd'))
            mask = read_mhd_image(os.path.join(sl_path, sl_name, phase+'_Mask.mhd'))
            image[mask == 0] = 0
            [ys, xs] = np.where(mask != 0)
            border = 0
            roi_image = image[np.min(ys) - border:np.max(ys)+border, np.min(xs)-border: np.max(xs)+border]
            save_path = os.path.join(roi_save_path, phase+'_ROI.mhd')
            save_mhd_image(roi_image, save_path)

# 将非RO的区域置为0I，这c里还会提取Expand之后的ROI
def ExtractSLExpandROIZero(sl_path, roi_save_dir):
    sl_names = os.listdir(sl_path)
    for sl_name in sl_names:
        print '-'*15, 'processing %s' % sl_name, '-'*15
        roi_save_path = os.path.join(roi_save_dir, sl_name)
        if not os.path.exists(roi_save_path):
            os.mkdir(roi_save_path)
        for phase in ['NC', 'ART', 'PV']:
            image = read_mhd_image(os.path.join(sl_path, sl_name, 'SignificantLayer', phase+'_Image.mhd'))[0]
            mask = read_mhd_image(os.path.join(sl_path, sl_name, 'SignificantLayer',  phase+'_Mask.mhd'))[0]
            expand_mask = read_mhd_image(os.path.join(sl_path, sl_name, 'SignificantLayer', phase+'_Mask_Expand.mhd'))[0]
            from copy import copy
            copy_image = copy(image)
            # print np.shape(copy_image)
            # print np.shape(expand_mask)
            copy_image[expand_mask == 0] = 0
            image[mask == 0] = 0
            [ys, xs] = np.where(mask != 0)
            border = 0
            roi_image = image[np.min(ys) - border:np.max(ys)+border, np.min(xs)-border: np.max(xs)+border]
            [ys, xs] = np.where(expand_mask != 0)
            roi_image_expand = copy_image[np.min(ys) - border:np.max(ys)+border, np.min(xs)-border: np.max(xs)+border]
            save_path = os.path.join(roi_save_path, phase+'_ROI.mhd')
            save_expand_path = os.path.join(roi_save_path, phase+'_ROI_Expand.mhd')
            save_mhd_image(roi_image, save_path)
            save_mhd_image(roi_image_expand, save_expand_path)

def ExtractSLROIS(sl_paths, roi_save_dir):
    states = ['category_by_level']
    types = ['1', '2', '3', '4', '5']
    for state in states:
        for cur_type in types:
            cur_sl_path = os.path.join(sl_paths, state, cur_type)
            roi_save_path = os.path.join(roi_save_dir, state, cur_type)
            ExtractSLExpandROIZero(cur_sl_path, roi_save_path)
            # show_ROI_size(cur_sl_path)

def show_ROI_size(sl_path):
    sl_names = os.listdir(sl_path)
    for sl_name in sl_names:
        size = []
        print '-'*15, 'processing %s' % sl_name, '-'*15
        for phase in ['NC', 'ART', 'PV']:
            mask = read_mhd_image(os.path.join(sl_path, sl_name, phase+'_Mask.mhd'))
            # image[mask == 0] = 0
            [ys, xs] = np.where(mask != 0)
            cur_size = [np.max(ys) - np.min(ys), np.max(xs) - np.min(xs)]
            size.append(cur_size)
        print sl_name, '->', size

ExtractSLROIS(
    '/home/give/Documents/dataset/LI-RADS/data/whole_data/',
    '/home/give/Documents/dataset/LI-RADS/data/roi_data/'
)

--------------- processing 3065890-2379159 ---------------
--------------- processing 3622154-3224937 ---------------
--------------- processing 2956711-2810792 ---------------
--------------- processing 2857450-2784382 ---------------
--------------- processing 1966940-2487518 ---------------
--------------- processing 3462780-3160156 ---------------
--------------- processing 3356280-2864468 ---------------
--------------- processing 1725960-2701744 ---------------
--------------- processing 8082242-2842685 ---------------
--------------- processing 1691221-2810735 ---------------
--------------- processing 2945085-2419111 ---------------
--------------- processing 938048-2637377 ---------------
--------------- processing 1834763-2841122 ---------------
--------------- processing 1398531-2584257 ---------------
--------------- processing 2956711-2739108 ---------------
--------------- processing 2871000-3178514 ---------------
--------------- processing 361278-2842930 ---------------

--------------- processing 3245102-2750207 ---------------
--------------- processing 2450847-1652689 ---------------
--------------- processing 3595077-3175407 ---------------
--------------- processing 3361967-2790988 ---------------
--------------- processing 2295132-3001460 ---------------
--------------- processing 3620182-3227676 ---------------
--------------- processing 3050330-2354675 ---------------
--------------- processing 670817-2449789 ---------------
--------------- processing 3067502-2381826 ---------------
--------------- processing 3534452-3079442 ---------------
--------------- processing 3480389-2988966 ---------------
--------------- processing 3357838-2785406 ---------------
--------------- processing 3080866-2403987 ---------------
--------------- processing 3415219-2871781 ---------------
--------------- processing 3318667-2720750 ---------------
--------------- processing 3372433-2806976 ---------------
--------------- processing 1464234-1951494 --------------

--------------- processing 3351671-2775790 ---------------
--------------- processing 1561985-2182384 ---------------
--------------- processing 2965287-2239225 ---------------
--------------- processing 3668105-3293461 ---------------
--------------- processing 3762947-3422066 ---------------
--------------- processing 3347183-2766992 ---------------
--------------- processing 2767306-1999418 ---------------
--------------- processing 2803559-2043272 ---------------
--------------- processing 3826689-3506892 ---------------
--------------- processing 3677388-3306990 ---------------
--------------- processing 3126862-2458412 ---------------
--------------- processing 3358635-2792688 ---------------
--------------- processing 2970918-2246507 ---------------
--------------- processing 2796945-2036079 ---------------
--------------- processing 3187050-2543573 ---------------
--------------- processing 8152526-3177106 ---------------
--------------- processing 2810769-2050257 -------------

In [8]:
def convert_format(roi_dir):
    '''
        将我们上述提取的ROI转化为代码相应的格式
    '''
    for level in [1, 2, 3, 4, 5]:
        cur_dir = os.path.join(roi_dir, str(level))
        for name in os.listdir(cur_dir):
            source = os.path.join(
                cur_dir, name
            )
            target = os.path.join(
                roi_dir, name+'_'+str(level)
            )
            shutil.copytree(
                source,
                target
            )

In [9]:
convert_format(
    '/home/give/Documents/dataset/LI-RADS/data/roi_data/Untitled Folder'
)

In [13]:
def statics_category(roi_dir):
    '''
        统计类别
    '''
    names = os.listdir(roi_dir)
    category_number = [0, 0, 0, 0, 0]
    for name in names:
        cur_level = int(name[-1])-1
        category_number[cur_level] += 1
    for i in range(5):
        print (i+1), '->', category_number[i]
statics_category('/home/give/Documents/dataset/LI-RADS/data/roi_data/2-class-train-val/train')

1 -> 37
2 -> 19
3 -> 18
4 -> 56
5 -> 116
